In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt

# --- CONFIG ---
input_csv = "22118_mag.csv"  # <-- Set your file path
output_csv = "binned_summary2.csv"
lat_col = "lat"
lon_col = "lon"

lithium_valley_coords =  [
  (-115.8, 33.0),
  (-115.4, 33.0),
  (-115.4, 33.4),
  (-115.8, 33.4),
  (-115.8, 33.0)
]

# --- LOAD DATA ---
df = pd.read_csv(input_csv)
df.columns = df.columns.str.strip()  # Remove accidental whitespace in headers





In [ ]:
from shapely.geometry import Polygon
from shapely import vectorized

lithium_valley_polygon = Polygon(lithium_valley_coords)

# REMOVED THE LV FILTERING 
mask = vectorized.contains(lithium_valley_polygon, df[lon_col].values, df[lat_col].values)
df_filtered = df#[mask].copy()
print(f"Filtered data shape: {df_filtered.shape}", 'ORIGINAL', df.shape)


C:\Users\ratth\AppData\Local\Temp\ipykernel_5536\3845983222.py:7: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask = vectorized.contains(lithium_valley_polygon, df[lon_col].values, df[lat_col].values)


Filtered data shape: (15862036, 27) ORIGINAL (15862036, 27)


In [ ]:


# --- BINNING ---
bin_size = 0.002  # Bin size found by a grid search to minimize the nulls while maximizing the granularity with the rounding integer as well. 
df_filtered['lat_bin'] = (df_filtered[lat_col] / bin_size).round(0) * bin_size
df_filtered['lon_bin'] = (df_filtered[lon_col] / bin_size).round(0) * bin_size

# --- AGGREGATION ---
group_cols = ['lat_bin', 'lon_bin']
numeric_cols = df_filtered.select_dtypes(include=[np.number]).columns.tolist()
agg_funcs = ['mean']# 'max', 'count'] # Mean is the standard practice, verified that the median and mean were about the same 
agg_dict = {col: agg_funcs for col in numeric_cols if col not in group_cols}
summary = df_filtered.groupby(group_cols).agg(agg_dict).reset_index()



summary.columns = [f"{col}_{stat}" if stat else f"{col}" for col, stat in summary.columns]
#summary.to_csv(output_csv, index=False)
#print(f"Binned and aggregated summary written to: {output_csv}")


value_cols = summary.columns #['comp_mag_mean',]  # Make sure this exists after aggregation


In [6]:
summary.to_csv('binned_geoflight.csv', index=False)


In [ ]:
import numpy as np
from scipy.stats import norm
import pandas as pd

### OLD EDA 
threshold =2 
not_plot = ['lon','lat','x','y','base','fid']
show = [
    'calc_radar_mean',
    'comp_mag_mean',
    'dc_lvl_mag_mean',
    'dem_mean',
    'diurnal_corrected_mag_mean',
    'drape_mean',
    'filtered_base_mean',
    'final_mag_mean',
    'igrf_base_cor_mean',
    'igrf_cor_mean',
    'micro_level_adjustment_mean',
    'micro_level_mag_mean',
    'micro_level_surface_mean',
    'radar_mean',
    'raw_mag_mean',
    'gps_elev_mean'
]

layers = [col for col in value_cols if col in show]
residuals_dict = {}

for value_col in layers:
    if value_col in summary.columns and "lat" not in value_col and "lon" not in value_col:
        # Pivot to grid
        pivot = summary.pivot_table(index='lat_bin', columns='lon_bin', values=value_col)

        # Residuals (Anomaly Enhancement)
        regional = pivot.rolling(window=15, min_periods=1, center=True).mean()
        residuals = pivot - regional

        # Normalize
        std = np.nanstd(residuals.values)
        norm_residuals = residuals / std if std != 0 else residuals

        # Flatten to long
        flat = norm_residuals.reset_index().melt(
            id_vars='lat_bin',
            var_name='lon_bin',
            value_name=f"{value_col}_stdnorm"
        )

        # Add percentile column
        flat[f"{value_col}_percentile"] = norm.cdf(flat[f"{value_col}_stdnorm"]) * 100

        # Filter: Only keep abs(stdnorm) > threshold and finite/notnull
        flat = flat[
            flat[f"{value_col}_stdnorm"].notnull() &
            np.isfinite(flat[f"{value_col}_stdnorm"]) &
            (flat[f"{value_col}_stdnorm"].abs() > threshold)
        ]

        # Merge into dict by key (lat_bin, lon_bin)
        for _, row in flat.iterrows():
            key = (row['lat_bin'], row['lon_bin'])
            if key not in residuals_dict:
                residuals_dict[key] = {'lat_bin': row['lat_bin'], 'lon_bin': row['lon_bin']}
            residuals_dict[key][f"{value_col}_stdnorm"] = row[f"{value_col}_stdnorm"]
            residuals_dict[key][f"{value_col}_percentile"] = row[f"{value_col}_percentile"]

# Create final DataFrame
df_final = pd.DataFrame(list(residuals_dict.values()))
df_final.sort_values(['lat_bin', 'lon_bin'], inplace=True)
df_final.to_csv('Geoflight_z2+_percentile.csv', index=False)
print(f"Saved {df_final.shape[0]} rows x {df_final.shape[1]} columns to Geoflight_z2+_percentile.csv")


Saved 39188 rows x 34 columns to Geoflight_z2+_percentile.csv


In [ ]:
for value_col in summary.columns:
    # Pivot to grid
    pivot = summary.pivot_table(index='lat_bin', columns='lon_bin', values=value_col)
    lats = pivot.index.values
    lons = pivot.columns.values

    # 1. Raw Mean Heatmap
    plt.figure(figsize=(12, 6))
    plt.title(f'Raw Mean Heatmap: {value_col}')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.imshow(
        pivot,
        aspect='auto',
        origin='lower',
        extent=[lons.min(), lons.max(), lats.min(), lats.max()],
        cmap='RdBu'
    )
    plt.colorbar(label=value_col)
    plt.tight_layout()
    plt.show()

    # 2. Residuals (Anomaly Enhancement)
    regional = pivot.rolling(window=15, min_periods=1, center=True).mean()
    residuals = pivot - regional

    plt.figure(figsize=(12, 6))
    plt.title(f'Residuals (Local Anomaly): {value_col}')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.imshow(
        residuals,
        aspect='auto',
        origin='lower',
        extent=[lons.min(), lons.max(), lats.min(), lats.max()],
        cmap='seismic'
    )
    plt.colorbar(label=f'{value_col} Residual')
    plt.tight_layout()
    plt.show()

    # 3. Z-Score Map (Standardized Anomalies)
    pivot_flat = pivot.stack()
    zscore = (pivot - pivot_flat.mean()) / pivot_flat.std()

    plt.figure(figsize=(12, 6))
    plt.title(f'Z-Score Map: {value_col}')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.imshow(
        zscore,
        aspect='auto',
        origin='lower',
        extent=[lons.min(), lons.max(), lats.min(), lats.max()],
        cmap='seismic'
    )
    plt.colorbar(label='Z-score')
    plt.tight_layout()
    plt.show()